### installing necessary packages

In [28]:
#!pip install chromadb google-cloud-bigquery pandas langchain google-cloud-aiplatform gradio

In [29]:
from google.cloud import bigquery
import pandas as pd
import requests
from google.oauth2 import service_account
import vertexai
from vertexai.language_models import TextGenerationModel


### we will be looking for data in public bigquery datasets

We're creating explicit list of tables to search for

In [30]:

bqclient = bigquery.Client(project='bigquery-public-data')

In [31]:
#project_ref = bqclient.project('bigquery-public-data')

table_list=[
    {'dataset': 'iowa_liquor_sales', 'table': 'sales'},
    {'dataset': 'london_bicycles', 'table': 'cycle_hire'},
    {'dataset': 'london_bicycles', 'table': 'cycle_stations'},
    {'dataset': 'ml_datasets', 'table': 'census_adult_income'},
    {'dataset': 'ml_datasets', 'table': 'credit_card_default'},
    {'dataset': 'ml_datasets', 'table': 'holidays_and_events_for_forecasting'},
    {'dataset': 'ml_datasets', 'table': 'iris'},
    {'dataset': 'ml_datasets', 'table': 'penguins'},
    {'dataset': 'ml_datasets', 'table': 'ulb_fraud_detection'},
    {'dataset': 'new_york_citibike', 'table': 'citibike_stations'},
    {'dataset': 'new_york_citibike', 'table': 'citibike_trips'},
    {'dataset': 'fdic_banks', 'table': 'institutions'},
    {'dataset': 'fdic_banks', 'table': 'locations'},
    {'dataset': 'fda_food', 'table': 'food_enforcement'},
    {'dataset': 'fda_food', 'table': 'food_events'},
    {'dataset': 'fcc_political_ads', 'table': 'broadcast_tv_radio_station'},
    {'dataset': 'fcc_political_ads', 'table': 'content_info'},
    {'dataset': 'fcc_political_ads', 'table': 'file_history'},
    {'dataset': 'fcc_political_ads', 'table': 'file_record'},
]



### We're using a variable to generate missing table descriptions using Text Bison
if True then generate missing table descriptions

In [32]:


GENERATE_TABLE_DESCRIPTIONS = True


### Function that sticks together dataset,table,field descriptions
if GENERATE_TABLE_DESCRIPTIONS is true then missing table descriptions are generated in bison based on other descriptions

In [33]:
def assemble_description(dataset_ref, table_ref,GENERATE_TABLE_DESCRIPTIONS):
    table=bqclient.get_table(table_ref)
    dataset=bqclient.get_dataset(dataset_ref)
    description=f"Dataset name: '{dataset_ref.dataset_id}', Dataset description:'{dataset.description}', Table name:\"{str(table_ref.table_id).replace('_',' ')}\", Table description:'{table.description}'"
    description=description + "Schema attributes: " + ",".join([f"column-name: {field.name}, column-type:\"{field.field_type}\" ,column-description: \"{field.description}\"" for field in table.schema])
    
    if table.description is None:
        returned_table_description="No description available"
    else:
        returned_table_description=table.description
        
    if GENERATE_TABLE_DESCRIPTIONS and table.description is None:
        parameters = {
            "temperature": 0.9,  # Temperature controls the degree of randomness in token selection.
            "max_output_tokens": 1000,  # Token limit determines the maximum amount of text output.
            "top_p": 0.8,  # Tokens are selected from most probable to least until the sum of their probabilities equals the top_p value.
            "top_k": 40,  # A top_k of 1 means the selected token is the most probable among all tokens.
        }
        model = TextGenerationModel.from_pretrained("text-bison@002")
        response = model.predict(
            "Please give brief description of a table that describes contents and purpose of the table for table users. Please do not describe or list attributes of a table only table general description. This is table schema:" + description,
            **parameters,
        )
        print(f"Response from Model: {response.text}")
        description=f"Dataset name: '{dataset_ref.dataset_id}', Dataset description:'{dataset.description}', Table name:\"{str(table_ref.table_id).replace('_',' ')}\", Table description:'{response.text}'"
        description=description + "Schema attributes: " + ",".join([f"column-name: {field.name}, column-type:\"{field.field_type}\" ,column-description: \"{field.description}\"" for field in table.schema])
        returned_table_description=response.text

    return returned_table_description,description

### Assemble table description
Calling the above function for each table


In [34]:

for i in table_list:
    
    
    dataset_ref = bqclient.dataset(i['dataset'])
    table_ref = dataset_ref.table(i['table'])
    dataset=bqclient.get_dataset(dataset_ref)
    table=bqclient.get_table(table_ref)
    #print(assemble_description(dataset_ref, table_ref))
    i['table_description'],i['description']=assemble_description(dataset_ref, table_ref,GENERATE_TABLE_DESCRIPTIONS)
    #print(dataset_ref,table_ref)

Response from Model:  **Table: cycle hire**

This table contains data about cycle hires in London. Each row represents a single hire. The data includes the duration of the hire, the bike ID, the bike model, the start and end dates and times, the start and end station IDs and names, and the end and start station logical terminal and priority IDs.

This data can be used to analyze cycle hire patterns in London, such as which stations are most popular, which times of day are busiest, and how long people typically rent bikes for. It can also be used to track the performance of the cycle hire scheme and to identify areas for improvement.
Response from Model:  The table 'cycle stations' in the 'london_bicycles' dataset contains information about bicycle stations in London. Each row represents a single station and provides details such as its unique ID, installation status, location (latitude and longitude), name, number of available bikes, number of available docks, number of empty docks, te

### Create embeddings from descriptions

In [35]:
import chromadb
from chromadb.utils import embedding_functions
import langchain
from langchain.embeddings import VertexAIEmbeddings
from vertexai.language_models import TextEmbeddingModel

In [36]:

chromaclient = chromadb.Client()
collection = chromaclient.get_or_create_collection("my_tables")


## Load embeddings into vector database

In [37]:
from chromadb import Documents, EmbeddingFunction, Embeddings
from chromadb.utils import embedding_functions
from langchain.vectorstores import Chroma
from langchain.embeddings import VertexAIEmbeddings


In [38]:

#VAIembeddings=VertexAIEmbeddings(model_name='textembedding-gecko@002')
#model = TextEmbeddingModel.from_pretrained("textembedding-gecko@002")


### We're not calling embedding functions ourselves. 
We point ChromaDB to use Vertex Embeddings every time new document is loaded or every time a query to DB is made

In [39]:

#for non english embeddings use model_name=textembedding-gecko-multilingual@001

db=Chroma(client=chromaclient,collection_name='my_tables',embedding_function=VertexAIEmbeddings(model_name='textembedding-gecko@001',task_type="SEMANTIC_SIMILARITY"))

In [40]:
from langchain.docstore.document import Document


for i in table_list:
    doc=Document(page_content=i['description'],metadata={'dataset': i['dataset'],'table': i['table'],'table_description' : i['table_description']},id=i['table'])
    db.add_documents( documents=[doc],                    
                     ids=[i['table']]
    )

## Query the vector store

db.similarity_search_with_relevance_scores(query="yummy", k=5, threshold=0.5, return_relevance_scores=True)

In [41]:
output=db.similarity_search_with_relevance_scores(query="bicycles", k=5, threshold=0.5, return_relevance_scores=True)

In [42]:
for i in output:
    print(i)

(Document(page_content='Dataset name: \'new_york_citibike\', Dataset description:\'None\', Table name:"citibike stations", Table description:\' The table contains information about Citi Bike stations in New York City. Each row represents a single station and provides information such as its unique identifier, name, location, rental methods accepted, number of available bikes and docks, and whether the station is currently installed, renting, or returning bikes. This information can be useful for planning bike trips, finding available stations, and understanding the overall status of the Citi Bike system.\'Schema attributes: column-name: station_id, column-type:"STRING" ,column-description: "Unique identifier of a station.",column-name: name, column-type:"STRING" ,column-description: "Public name of the station.",column-name: short_name, column-type:"STRING" ,column-description: "Short name or other type of identifier, as used by the data publisher.",column-name: latitude, column-type:"

In [43]:
import gradio as gr



def search_items(query):
    # Your search logic here
    # Return the search results
    
    # For example, let's assume we have a list of items

    # Filter the items based on the query
    if query =="":
        query="banks are bad"
    results = db.similarity_search_with_relevance_scores(query=query, k=5, threshold=0.5, return_relevance_scores=True)
    output=[]
    for i in results:
        output.append(i[0].metadata['dataset']+"."+i[0].metadata['table'])
        output.append(i[0].metadata['table_description'])
    update_show=[gr.Text(visible=True,value=x) for x in output]

    return update_show

result_tables_list = []


with gr.Blocks() as demo:
    search_phrase = gr.Textbox(label="Search phrase",placeholder="Bicycles")

    with gr.Column():
        for i in range(5):
            with gr.Row():
                table_field=gr.Text(show_label=False,visible=False)
                description_field=gr.Textbox(show_label=False,visible=False)
                result_tables_list.append(table_field)
                result_tables_list.append(description_field)
    search_phrase.change(search_items, search_phrase, result_tables_list)
    greet_btn = gr.Button("Search")

    greet_btn.click()
    greet_btn.click(search_items,search_phrase,result_tables_list)
        

demo.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
